In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# Lab08: Classification Continued
Welcome to lab08. We will be continuing our discussion of classification with K-Nearest Neighbors as well as Nearest Neighbors in more than 2 dimensions.


Relevant chapters from the textbook are:
* [Chapter 7](https://www.inferentialthinking.com/chapters/07/Visualization.html)
* [Chapter 8](https://www.inferentialthinking.com/chapters/08/Functions_and_Tables.html)
* [Chapter 17](https://www.inferentialthinking.com/chapters/17/Classification.html)



In [295]:
import numpy as np
from datascience import *
#from NearestNeighbors import *
import random 
# These lines set up graphing capabilities.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
np.random.seed(3)
random.seed(3)


# Ignore these Functions
def standard_units(x):
    return (x - np.mean(x))/np.std(x)

def make_grid(min_, max_, step):
    x_array = make_array()
    y_array = make_array()
    for x in np.arange(min_, max_, step):
        for y in np.arange(min_, max_, step):
            x_array = np.append(x_array, x)
            y_array = np.append(y_array, y)
    return x_array, y_array

def distance_nn(point1, point2):
    """The distance between two arrays of numbers."""
    return np.sqrt(np.sum((point1 - point2)**2))

def all_distances(training, point):
    """The distance between p (an array of numbers) and the numbers in row i of attribute_table."""
    attributes = training.drop('Class')
    def distance_from_point(row):
        return distance_nn(point, np.array(row))
    return attributes.apply(distance_from_point)

def table_with_distances(training, point):
    """A copy of the training table with the distance from each row to array p."""
    return training.with_column('Distance', all_distances(training, point))

def closest(training, point, k):
    """A table containing the k closest rows in the training table to array p."""
    with_dists = table_with_distances(training, point)
    sorted_by_distance = with_dists.sort('Distance')
    topk = sorted_by_distance.take(np.arange(k))
    return topk

def majority(topkclasses):
    """1 if the majority of the "Class" column is 1s, and 0 otherwise."""
    ones = topkclasses.where('Class', are.equal_to(1)).num_rows
    zeros = topkclasses.where('Class', are.equal_to(0)).num_rows
    if ones > zeros:
        return 1
    else:
        return 0

def classify(training, p, k):
    """Classify an example with attributes p using k-nearest neighbor classification with the given training table."""
    closestk = closest(training, p, k)
    topkclasses = closestk.select('Class')
    return majority(topkclasses)

def classify_table(training, points, k):
    """Classify a table of unlabled points using KNN"""
    def classify_p(p):
        return classify(training, p, k)

    classes = points.apply(classify_p)
    return points.with_column('Class', classes)

def draw_nearest_neighbor(table, point):
    with_distances = table_with_distances(table, point)
    distances_sorted = with_distances.sort('Distance', descending=False)

    t = distances_sorted.drop('Distance')

    x_closest = t.row(0).item(0)
    y_closest = t.row(0).item(1)

    table.scatter(table.labels[0], table.labels[1], group='Class')
    plt.scatter(point.item(0), point.item(1), color='red', s=30)
    plt.plot(make_array(point.item(0), x_closest), make_array(point.item(1), y_closest), color='k', lw=2);

# 0. 3D distance
In lab section:
1. Derive the formula for 3d distance. 
2. Give the formula (without proof) for N-d distance.


<!--
BEGIN QUESTION
name: q0
manual: false
-->
For submission:

**q0** What is the distance between the points $P_1=(1,1,1)$ and $P_2 = (4, 5, 13)$?

In [296]:
distance = ...

In [ ]:
grader.check("q0")

# 1. Lobsters
We are going to take another look at the lobster dataset. Recall that this meticulously handcrafted dataset contains information about claw width, tail lenght and the sex of lobsters. A value of zero in the 'Class' column denotes a female and a value of one denotes a male. 

In [298]:
lobster = Table.read_table('lobster.csv')
lobster

In [299]:
lobster.scatter('Claw Width', 'Tail Length', group='Class')


## 1.1 Visualizing the Lobster Decision Boundary 

Like last week, we are going to visualize the decision boundary produced by the Nearest Neighbors algorithm. This time, we have given you the code that generates these plots. The first step is to make a grid that contains points that fills the space.

Below, we make a table that contains such a grid using the `make_grid` function, which we have implemented for you. The  blue and yellow points are from the original dataset and the red points make up the grid.

In [300]:
x_array, y_array = make_grid(4, 20, .5)
test_grid = Table().with_columns(
    'Claw Width', x_array,
    'Tail Length', y_array
)
test_grid.scatter('Claw Width', 'Tail Length', color='red', alpha=0.4, s=30)


plt.scatter(lobster.column('Claw Width',), lobster.column('Tail Length'), c=lobster.column('Class'), edgecolor='k')

plt.xlim(4, 20)
plt.ylim(4, 20);

Next, we classify every point in the grid. We have implemented for you a function called `classify_table`, which labels every single point in an unlabled grid. Under the hood, this calls Nearest Neighbors on each row using `apply`. 

This may take a while because there are a lot of points to classify. (If you see the '*' in the brackets to the left of the cell, the code is still running).

In [301]:
labeled_grid = classify_table(lobster, test_grid, 1)



Now we plot the result.

In [302]:
labeled_grid.scatter('Claw Width', 'Tail Length', group='Class', alpha=0.4, s=30)

plt.scatter(lobster.column('Claw Width'), lobster.column('Tail Length'), c=lobster.column('Class'), edgecolor='k')

plt.xlim(4,20)
plt.ylim(4,20)

## 1.2 A Problem with Nearest Neighbor Search
The regions above identifying female and male lobsters are not cleanly separated. Let's investigate what is causing the issue by looking at a single point: (17,5)

We have given you a function called `draw_nearest_neighbor` that takes in an arbitrary table and an arbitrary point and plots the points in the table, the point, and a line between the point (in red) and the closest point to it in the table. 


In [303]:
draw_nearest_neighbor(lobster, make_array(17,5))

We arguably want to see the region of male class '1' lobsters be a solid diagonal stripe. It would also be nice if the regions of female class '0' lobsters made solid triangles on either side of the stripe. However, because some male lobsters cross into the region of female lobsters, and because our classified data points are relatively sparse, some points well within the expected region of the female lobsters have as their nearest neighbor a male lobster.

We can see this occured witht the point (17, 5). You may notice, that the next several closest neighbors of that point are all female. This suggests that we can generate a better decision boundary by looking at the labels of a few closest neighbors instead of just one.

# 2. K Nearest Neighbors
K Nearest Neighbors is a classification algorithm that classifies a point based on whatever the majority of the point's k nearest-neighbors are labeled, where k is an arbitrary integer. 

## 2.1 Implementing K Nearest Neighbors
The implementation of the K Nearest Neighbors algorithm is very similar to the regular Nearest neighbors algorithm.

1. We find the distance between the unlableled point and every point in our table. 
2. We sort the table by distances. 
3. Instead of taking the row with the smallest distance, we will look at the top-k rows with the smallest distances.
4. We find the majority class of those rows

We have implemented steps #1 and #2 for you because we did those steps in lab07. The function `table_with_distances` takes in a table with 3 columns, one of which is called 'Class', and a point and returns a new table with an additional column called 'Distances' which has the distance between each row and the given point. 


<!--
BEGIN QUESTION
name: q2_1
points: 1
manual: false
-->

**q2.1** We will now implement step #3. Write a function called `closest` that takes in a labeled table, a point, and a value k and returns the top-k closest points. 

you may use`table_with_distances`.

In [304]:
def closest(input_table, point, k):
    # Recommended code structure
    distance_table = ...
    sorted_table_with_distances = ...
    topk = ...
    return topk
    

# To check your solution:
kclosest = closest(lobster, make_array(12.55,18.6), 4)
kclosest

In [ ]:
grader.check("q2_1")

### 2.2 Finding the Majority
<!--
BEGIN QUESTION
name: q2_1_2
points: 1
manual: false
-->
The last step is to find the majority label from the table of k-closet neighbors.

**q2.2** Write a function called `majority` that takes in a table with a 'Class' column that has values of either 0 or 1.  The function should return the class that occurs with the highest frequency.

To do this, you will first need to find the number of times each label occurs. Then you will have to use an if statement to return the correct label

In [307]:
def majority(table):
    ...

In [ ]:
grader.check("q2_1_2")

To check your solution:

In [311]:
majority_label = majority(kclosest)
majority_label

## 2.3 K-Nearest Neighbors Function
**q2.3** Now that we have implemented all of the components, we can stitch them together to create a `classify-knn` function. `classify-knn` should take in a table, a point, and a number k, and return the class assigned to the majority of the k-clsoest neighbors of the point.

If you use `closest` and `majority` correctly, your code should be fairly short.
<!--
BEGIN QUESTION
name: q2_3
points: 1
manual: false
-->

In [312]:
def classify_knn(input_table, point, k):
    ...

In [ ]:
grader.check("q2_3")

We have implemented for you a more general function that can classify an entire table of unlabled points. This function requires your `classify_knn` function to work correctly.

Marvel at the fact that you define functions inside functions, but not for too long. This is necessary because we have to make a new `classify_knn` function for every unlabeled point if we want to use it with `apply`.

In [317]:
def classfiy_table_knn(labeled_table, unlabeled_table, k):
    
    def classify_point_by_table_k(unlabeled_point):
        return classify_knn(labeled_table, unlabeled_point, k)
    
    labels = unlabeled_table.apply(classify_point_by_table_k)
    table_with_labels = unlabeled_table.with_column('Class', labels)
    
    return table_with_labels

## 2.4 KNN Decision Boundary

Now that we have a function that can classify an entire table using KNN, let's look at the new decision boundary looks like. We will run the same grid-classification experiments as before, but we will now classify all points using K Nearest Neighbors with an aribitrarily chosen value of k=5.

Again, the first cell may take a while to run because there are a lot of points in the grid to classify.

In [318]:
x_array, y_array = make_grid(4, 20, .5)
test_grid = Table().with_columns(
    'Claw Width', x_array,
    'Tail Length', y_array
)
k=5
labeled_grid = classfiy_table_knn(lobster, test_grid, 5)

In [319]:
labeled_grid.scatter('Claw Width', 'Tail Length', group='Class', alpha=0.4, s=30)

plt.scatter(lobster.column('Claw Width'), lobster.column('Tail Length'), c=lobster.column('Class'), edgecolor='k')

plt.xlim(4,22)
plt.ylim(4,22)

<!-- BEGIN QUESTION -->


<!--
BEGIN QUESTION
name: q2_4
points: 1
manual: true
-->
Compare the decision boundary of the table above with the decision boundary displayed in section 2.1. Which do you prefer and why?

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# 2.5 Brain Teasers

<!-- BEGIN QUESTION -->

Two more brain teasers about K Nearest Neighbors
<!--
BEGIN QUESTION
name: q2_5_1
points: 1
manual: true
-->
**q2.5.1** In general, datascientists choose k to be odd. Why do you think this is?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q2_5_2
points: 1
manual: false
-->
**q2.5.2** What value of k makes K-Nearest Neighbors equivalent to vanilla Nearest Neighbor Search as we impleneted in lab07?

In [320]:
k=...
    

In [ ]:
grader.check("q2_5_2")

# 2.6 Evaluating Error

We will now evaluate the accuracy by comparing the predictions of our algorithm on the testing set against their actual labels. 

The code below splits the data into a training and testing set and creates an array that represents the *predicted* class of each point in the test dataset using Nearest Neighbors. We visualized the decision boundary of the training this training set with k=1 in lab07 section 5.1

In [322]:
training = lobster.take(np.arange(160))
testing = lobster.take(np.arange(160, 200))

test_predictions = classify_table(training, testing.drop('Class'), 1).column('Class')
test_predictions

The **accuracy** of a model is defined as 
$$\frac{\text{Correct Predictions}}{\text{Total Predictions}}$$

<!--
BEGIN QUESTION
name: q2_6
manual: false
-->

**q2.6** What is the accuracy of the Nearest Neighbors Classifier on the *test* data?

There are a few ways you could solve this problem. One way is to:
1. Make an array of the actual classes of the dataset by getting the 'Class' column of `testing`.
2. Take the difference between the predicted and actual labels by subtracting the two arrays.
3. Use the result of that array and the function np.count_nonzero() to find the number of points classified incorrectly.
4. Use step (3) to calculate the number of points calculated correctly.
5. Divide by the total number of points.

In [323]:
# Suggested code structure
true_classes = ...
diff = ...
num_incorrect = ...
num_correct = ...
accuracy = ...
accuracy

In [ ]:
grader.check("q2_6")

# 3 More Than Two Dimensions

We are going to look at a new task, predicting whether or not a dollar bill is fraudulent. To do this, we will use the `banknote` dataset. Each row in `banknote` gives information about an image of a bill. This information includes four features that were computed by researches using methods we do not need to understand. The fifth 'Class' column tells us whether the the bill is counterfit (one) or real (zero).

In [325]:
banknotes = Table.read_table('banknote.csv')
banknotes

Let's plot the first two columns and see what these features tell us.

In [326]:
banknotes.scatter('WaveletVar', 'WaveletCurt', group='Class')

These features distinguish the two classes to an extent, but there is still a significant region of overlap on which these features are not enough for classification. 

Make scatter plots with other columns to see if other pairs of features are more decisive. Feel free to add more cells.

In [327]:
# UNCOMMENT
# banknotes.scatter(..., ..., group='Class')

You should find that new two features are quite enough. So let's try three features instead. The code below makes a 3D scatter plot based off of 'WaveletSkew', 'WaveletVar' and 'WaveletCurt'.

In [328]:
ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')
ax.scatter(banknotes.column('WaveletSkew'), 
           banknotes.column('WaveletVar'), 
           banknotes.column('WaveletCurt'), 
           c=banknotes.column('Class'));

By selecting three features to plot, we have found a much more decisive boundary between the two classes. The clear clustering of each class should make it apparent that we can use K-Nearest Neighbors on this dataset. We will use the same setup as before. Note that of all the code we have written so far, the only piece that needs changing is the distance function we wrote at the beggining of lab 7.  

Recall from section 0 of this lab, that the distance function can be written for an arbitrary number of dimensions. This is how we implemented the distance function in `classify_table`. So we don't actually need to change anything. 

## 3.1 Training and Testing
Before we do anything, we need to split our data into training and testing datasets. We have to be careful, because unlike the `lobster` dataset, `banknotes` is not shuffled. If we select the first 100 rows for the trainig set, all rows will have the same class. 

<!--
BEGIN QUESTION
name: q3_1a
points: 1
manual: false
-->
**q3.1a** Make a shuffled version of `banknotes`. Then make the training set be the first 50 rows of that table and make the test set be the next 30 rows. We will use the Table function `sample` that will allow us to randomly select rows with replacement. To make sure that you read these instructions, here's the code that you need to assign to shuffled_banknotes: `banknotes.sample(with_replacement=False)`.

In [329]:
shuffled_banknotes = ...
banknotes_training = ...
banknotes_testing = ...

In [ ]:
grader.check("q3_1a")

<!--
BEGIN QUESTION
name: q3_1b
points: 3
manual: false
-->
**q3.1b** Write a function called `accuracy` that returns the accuracy of a model given an array of the models predictions and an array of the true labels. You may want to refer back to question 2.6. (This time we are giving you the array of predictions as a parameter, so the code structre is a little simpler.)

In [331]:
def accuracy(predictions, labels):
    # Suggested code structure
    diff = ...
    num_incorrect = ...
    num_correct = ...
    accuracy = ...
    return accuracy

In [ ]:
grader.check("q3_1b")

# 3.2 Comparing Different Values of K
Finally, let's use our `accuracy` function to evaluate different sizes of the neighboorhhood for evaluation in KNN. First, run the code following code which gives the arrays of predictions for each value of k stored in a list.

In [335]:
predictions_1 = classify_table(banknotes_training, banknotes_testing.drop('Class'), 1).column('Class')
predictions_3 = classify_table(banknotes_training, banknotes_testing.drop('Class'), 3).column('Class')
predictions_5 = classify_table(banknotes_training, banknotes_testing.drop('Class'), 5).column('Class')
predictions_7 = classify_table(banknotes_training, banknotes_testing.drop('Class'), 7).column('Class')


**q3.2** Now find the accuracy of each models' predictions using your `accuracy` function.
<!--
BEGIN QUESTION
name: q3_2
points: 3
manual: false
-->

In [336]:
accuracy_1 = 
accuracy_3 = 
accuracy_5 = 
accuracy_7 = 

In [ ]:
grader.check("q3_2")

<!-- BEGIN QUESTION -->

**q3.3** Explain the trends you see in accuracy as $k$ increases. Do you see anything that you did not expect? What do you think might explain what you are seeing?
<!--
BEGIN QUESTION
name: q3_3
points: 3
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



Congratulations, you are finished with this lab!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export("lab08.ipynb")